# Compiling the post mass transfer systems that contain a Be star

 Be+sdOB binaries are particularly valuable for studying mass transfer, as they represent clear examples of past binary interaction

In [94]:
import numpy as np
import pandas as pd
import re
import subprocess
import ast



## Papers of interest :

- **Lechien + 2025** compute the mass transfer efficiency "Binary stars take what they get: Evidence for Efficient Mass Transfer from Stripped Stars with Rapidly Rotating Companion" [2025arXiv250514780L](https://ui.adsabs.harvard.edu/abs/2025arXiv250514780L/abstract)


They combine data from: 

- **Wang + 2021** "The Detection and Characterization of Be+sdO Binaries from HST/STIS FUV Spectroscopy" [2021AJ....161..248W](https://ui.adsabs.harvard.edu/abs/2021AJ....161..248W/abstract)

- **Klement + 2024** The CHARA Array Interferometric Program on the Multiplicity of Classical Be Stars: New Detections and Orbits of Stripped Subdwarf Companions  [2024ApJ...962...70K](https://ui.adsabs.harvard.edu/abs/2024ApJ...962...70K/abstract)

- **Klement + 2025**  VLTI/GRAVITY enables the determination of the first dynamical masses of a classical Be + stripped and bloated pre-subdwarf binary [2025A&A...694A.208K](https://ui.adsabs.harvard.edu/abs/2025A%26A...694A.208K/abstract)




## Read the Latex table

In [167]:
### READING the compiled table from Lechien et al. 2025

# Read the LaTeX file
with open("../data/Lechien25_Table1.tex", "r") as f:
    lines = f.readlines()

# Find the start and end of the tabular environment
table_lines = []
in_table = False
for line in lines:
    if r"\begin{tabular}" in line:
        in_table = True
        continue
    if r"\end{tabular}" in line:
        break
    if in_table:
        table_lines.append(line.strip())

# Filter out lines we don't need
table_rows = [line for line in table_lines if not line.startswith("\\") and line]

# Now split and clean each row
cleaned_rows = []
for row in table_rows:
    row = row.split("&")
    row = [re.sub(r"\\\w+", "", col).strip() for col in row]  # remove LaTeX commands like \cite
    row = [re.sub(r"\$|\\", "", col) for col in row]          # remove $ signs and backslashes
    row = [col.replace("pm", "±").replace("{", "").replace("}", "") for col in row]
    cleaned_rows.append(row)

# Create a DataFrame (skip header rows if needed)
header = [
    "HD", "Name", "M_Be", "M_sdOB", "Period", "Reference", "Tier",
    "beta_min", "beta_range"
]

# Trim extra rows that aren't data
data_rows = [r for r in cleaned_rows if len(r) == len(header)]

Lechien25 = pd.DataFrame(data_rows, columns=header)

# Remove the first 2 rows (which arent data)
Lechien25 = Lechien25.iloc[2:].reset_index(drop=True)

display(Lechien25)  

,HD,Name,M_Be,M_sdOB,Period,Reference,Tier,beta_min,beta_range
0,58978,FY CMa,10--13,1.1--1.5,37.253 0.007,peters_detection_2008,-,0.57,0.81 - 1.00
1,200120,59 Cyg,6.3--9.4,0.62--0.91,28.1871 0.0011,peters_far-ultraviolet_2013,-,0.50,0.75 - 1.00
2,10516,Per,9.6 0.3,1.2 0.2,126.6982 0.0035,mourard_spectral_2015,**,0.36,0.56 - 1.00
3,55606,-,6.0--6.6,0.83--0.90,93.76 0.02,chojnowski_remarkable_2018,-,0.06,0.23 - 0.83
4,109387,Dra,3.65 0.48,0.426 0.043,61.5496 0.0058,klement_dynamical_2022,*,0.05,0.25 - 0.83
5,113120,LS Mus,10.1 2.2,1.43 0.31,181.54 0.11,wang_orbital_2023,-,0.27,0.46 - 1.00
6,137387,Aps,11.8 1.0,1.60 0.14,192.1 0.1,wang_orbital_2023,-,0.41,0.62 - 1.00
7,152478,V846 Ara,6.5 1.3,0.53 0.11,236.50 0.18,wang_orbital_2023,-,0.64,0.94 - 1.00
8,157042,Ara,10.5 2.9,1.06 0.29,176.17 0.04,wang_orbital_2023,-,0.64,0.88 - 1.00
9,41335,HR 2142,17.6 5.7,1.03 0.22,80.8733 0.0044,klement_chara_2024,*,(1.00),(1.00 - 1.00)


## RA and DEC in the right format

In [168]:
# Get RA and DEC using the Get_Coords function
Lechien25['name'] = 'HD' + Lechien25['HD'].astype(str)

display(Lechien25['name'].values)
targets = Lechien25['name'].values.tolist()

# Run the script and capture output
result = subprocess.run(["python3", "Get_Coords_From_SIMBAD.py"] + targets, capture_output=True, text=True)

# Get the printed output as a string
RA_DEC = result.stdout

entries = re.findall(r'([A-Z0-9_]+)\s*=\s*({.*?})', RA_DEC, re.DOTALL)
RADEC_data = {}

for name, dict_str in entries:
    RADEC_data[name] = ast.literal_eval(dict_str)

RADEC_data.keys()
# Now `RADEC_data` is a Python dictionary with the object data
# print(RADEC_data['HD__58978']['RA'])
# print(RADEC_data['HD__58978']['Dec'])

# Extract RA and Dec values
RA_values = [RADEC_data[key]['RA'] for key in RADEC_data.keys() ]
DEC_values = [RADEC_data[key]['Dec'] for key in RADEC_data.keys() ]

# print(RA_values)
# print(DEC_values)   

array(['HD58978', 'HD200120', 'HD10516', 'HD55606', 'HD109387',
       'HD113120', 'HD137387', 'HD152478', 'HD157042', 'HD41335',
       'HD161306', 'HD183537', 'HD191610', 'HD194335', 'HD200310',
       'HD167128'], dtype=object)

### Put the periods in the right format

In [169]:
##############################
# Periods
Lechien25[['Period_val', 'Period_err']] = Lechien25['Period'].str.split(expand=True).astype(float)

Lechien25['Period_parsed'] = Lechien25.apply(
    lambda row: [row['Period_err'], row['Period_val'], row['Period_err']],
    axis=1
)


### Put the masses in the right format

In [170]:
##############################
# Masses
def parse_mass_entry(entry):
    entry = str(entry).strip()
    
    if '--' in entry:
        # Range case: "10--13"
        try:
            low, high = map(float, entry.split('--'))
            median = (low + high) / 2
            err = (high - low) / 2
            return [err, median, err]
        except ValueError:
            return None
    elif ' ' in entry:
        # Symmetric error case: "9.6 0.3"
        try:
            value, err = map(float, entry.split())
            return [err, value, err]
        except ValueError:
            return None
    else:
        # Single number, no error
        try:
            value = float(entry)
            return [None, value, None]
        except ValueError:
            return None

Lechien25['M_Be_parsed'] = Lechien25['M_Be'].apply(parse_mass_entry)
Lechien25['M_sdOB_parsed'] = Lechien25['M_sdOB'].apply(parse_mass_entry)

# Show the result
# display(Lechien25)

display(Lechien25[['M_Be_parsed', 'M_sdOB_parsed']])

,M_Be_parsed,M_sdOB_parsed
0,"[1.5, 11.5, 1.5]","[0.19999999999999996, 1.3, 0.19999999999999996]"
1,"[1.5500000000000003, 7.85, 1.5500000000000003]","[0.14500000000000002, 0.765, 0.14500000000000002]"
2,"[0.3, 9.6, 0.3]","[0.2, 1.2, 0.2]"
3,"[0.2999999999999998, 6.3, 0.2999999999999998]","[0.03500000000000003, 0.865, 0.03500000000000003]"
4,"[0.48, 3.65, 0.48]","[0.043, 0.426, 0.043]"
5,"[2.2, 10.1, 2.2]","[0.31, 1.43, 0.31]"
6,"[1.0, 11.8, 1.0]","[0.14, 1.6, 0.14]"
7,"[1.3, 6.5, 1.3]","[0.11, 0.53, 0.11]"
8,"[2.9, 10.5, 2.9]","[0.29, 1.06, 0.29]"
9,"[5.7, 17.6, 5.7]","[0.22, 1.03, 0.22]"


## Mass ratio

In [171]:
# Mass ratio values m2/m1
#  "M1": Lechien25['M_Be_parsed'],
# "M2": Lechien25['M_sdOB_parsed'],

def compute_q(m2_triplet, m1_triplet):
    # Unpack values
    m2_err_low, m2_val, m2_err_high = m2_triplet
    m1_err_low, m1_val, m1_err_high = m1_triplet
    
    # Nominal q value
    q_val = m2_val / m1_val
    
    # Propagate upper and lower errors separately
    # Lower error: (M2 - err-) / (M1 + err+) → gives lower q
    q_low = (m2_val - m2_err_low) / (m1_val + m1_err_high)
    err_low = q_val - q_low
    
    # Upper error: (M2 + err+) / (M1 - err-) → gives upper q
    q_high = (m2_val + m2_err_high) / (m1_val - m1_err_low)
    err_high = q_high - q_val

    return [err_low, q_val, err_high]


Lechien25['q'] = Lechien25.apply(
    lambda row: compute_q(row['M_sdOB_parsed'], row['M_Be_parsed']),
    axis=1
)

print(Lechien25['q'])


0     [0.028428093645484945, 0.11304347826086956, 0....
1     [0.03149478249085243, 0.09745222929936306, 0.0...
2     [0.02398989898989899, 0.125, 0.025537634408602...
3     [0.011544011544011551, 0.1373015873015873, 0.0...
4     [0.023976251285283093, 0.11671232876712329, 0....
5     [0.0505272478467359, 0.14158415841584157, 0.07...
6     [0.021530720338983053, 0.13559322033898305, 0....
7     [0.027692307692307697, 0.08153846153846155, 0....
8     [0.043489694385216784, 0.10095238095238096, 0....
9     [0.023758778774873188, 0.05852272727272727, 0....
10    [0.017175233298770548, 0.1302325581395349, 0.0...
11    [0.010226365546218502, 0.11223529411764706, 0....
12    [0.04911659417221603, 0.12140575079872205, 0.0...
13    [0.03831835686777921, 0.16463414634146345, 0.0...
14    [0.02634279566332863, 0.13488372093023254, 0.0...
15    [0.010272651876425454, 0.06367924528301887, 0....
Name: q, dtype: object


## Fix the references into a list of strings

In [172]:
# 
# np.unique(Lechien25['Reference'])

# Ref_dict = {'chojnowski_remarkable_2018':'2018ApJ...865...76C' ,
# 'klement_chara_2024': '2024ApJ...962...70K',
# 'klement_dynamical_2022':'2022ApJ...940...86K' ,
# 'klement_vltigravity_2025': '2025A&A...694A.208K',
# 'mourard_spectral_2015':'2015A&A...577A..51M',
# 'peters_detection_2008': '2008ApJ...686.1280P',
# 'peters_far-ultraviolet_2013': '2013ApJ...765....2P',
# 'wang_orbital_2023': '2023AJ....165..203W'
# }

# Map references to bibcodes
Ref_dict = {
    'chojnowski_remarkable_2018': '2018ApJ...865...76C',
    'klement_chara_2024': '2024ApJ...962...70K',
    'klement_dynamical_2022': '2022ApJ...940...86K',
    'klement_vltigravity_2025': '2025A&A...694A.208K',
    'mourard_spectral_2015': '2015A&A...577A..51M',
    'peters_detection_2008': '2008ApJ...686.1280P',
    'peters_far-ultraviolet_2013': '2013ApJ...765....2P',
    'wang_orbital_2023': '2023AJ....165..203W'
}

# Map each reference and wrap in a list
Lechien25['Reference_parsed'] = Lechien25['Reference'].map(lambda ref: [Ref_dict.get(ref, 'UNKNOWN')])


## Eccentricity
Table 5 from [Klement 2024](https://iopscience.iop.org/article/10.3847/1538-4357/ad13ec)

--> Essentially everything is zero except for:

- **HR 6819** The orbit is slightly eccentric, with e = 0.0289 ± 0.0058, and the semimajor axis of the orbit is 0.3800 ± 0.0093 AU. (klement_vltigravity_2025)

- **60 Cyg == HD 20031060** exhibits the highest orbital eccentricity at 0.20 ± 0.01, (Lechien 25)



In [175]:
# Define the orbital data from Table 5 in the paper
data = {
    "System": ["28 Cyg", "V2119 Cyg", "60 Cyg", "HR 2142", "HD 161306", "7 Vul"],
    "HD": [191610, 194335, 200310, 41335, 161306, 183537],
    "Eccentricity": [0.0, 0.0, 0.20, 0.0, 0.0, 0.0],
    "Eccentricity_Error": [0.0, 0.0, 0.01, 0.0, 0.0, 0.0]
}

# Create the DataFrame
eccentricities_df = pd.DataFrame(data)
display(eccentricities_df)

display(Lechien25[['HD', 'name', 'Name']])

,System,HD,Eccentricity,Eccentricity_Error
0,28 Cyg,191610,0.0,0.00
1,V2119 Cyg,194335,0.0,0.00
2,60 Cyg,200310,0.2,0.01
3,HR 2142,41335,0.0,0.00
4,HD 161306,161306,0.0,0.00
5,7 Vul,183537,0.0,0.00


,HD,name,Name
0,58978,HD58978,FY CMa
1,200120,HD200120,59 Cyg
2,10516,HD10516,Per
3,55606,HD55606,-
4,109387,HD109387,Dra
5,113120,HD113120,LS Mus
6,137387,HD137387,Aps
7,152478,HD152478,V846 Ara
8,157042,HD157042,Ara
9,41335,HD41335,HR 2142


# Combine this into the final data format!!

In [178]:
# Define columns, with key quantities stored as lists: [err-, value, err+]
columns = [
    "System Name", "RA", "Dec", "Period", "Eccentricity",
    "M1","M1_sin3i", "M2", "M2_sin3i", "q", "Mass Function",
    "Type1", "Type2", "Detection Method", "Reference", "Notes"
]


# Initialize empty DataFrame
Be_sdOB_table = pd.DataFrame(columns=columns)

Be_sdOB_table = pd.DataFrame({
    "System Name": Lechien25['name'],
    "RA": RA_values,
    "Dec": DEC_values,
    
    "Period": Lechien25['Period_parsed'], #Lechien25['Period'],
    "Eccentricity": [[np.nan, 0.0, np.nan]] * len(Lechien25), #PLACEHOLDER
    "M1": Lechien25['M_Be_parsed'],
    "M1_sin3i": [[np.nan, np.nan, np.nan]] * len(Lechien25), # No values, so fill with NaN
    "M2": Lechien25['M_sdOB_parsed'],
    "M2_sin3i": [[np.nan, np.nan, np.nan]] * len(Lechien25), # No values, so fill with NaN
    "q": Lechien25['q'] , #PLACEHOLDER
    "Mass Function": [[np.nan, np.nan, np.nan]] * len(Lechien25), # No values, so fill with NaN
    "Type1": ['Be']  * len(Lechien25),
    "Type2": ['sdOB']  * len(Lechien25),
    "Detection Method": ['RV'] * len(Lechien25),  # Assuming all are RV detections
    "Reference": Lechien25['Reference_parsed'],  # ADS bibcodes
    "Notes": ['q errors might be wrong'] * len(Lechien25),  # Placeholder for reference
})


#####
# Fix the few nonzero eccentricities
Be_sdOB_table.loc[Be_sdOB_table["System Name"] == "HD6819", "Eccentricity"] = [[0.0058, 0.0289, 0.00581]]
Be_sdOB_table.loc[Be_sdOB_table["System Name"] == "HD200310", "Eccentricity"] = [[0.01, 0.20, 0.01]]


In [183]:
display(Be_sdOB_table)

# Save the DataFrame to a pickle file
Be_sdOB_table.to_pickle("../result_tables/Be_sdOB_table.pkl")

,System Name,RA,Dec,Period,Eccentricity,M1,M1_sin3i,M2,M2_sin3i,q,Mass Function,Type1,Type2,Detection Method,Reference,Notes
0,HD58978,"[7.85e-06, 111.747845, 7.85e-06]","[1.247e-05, -23.086025, 1.247e-05]","[0.007, 37.253, 0.007]","[nan, 0.0, nan]","[1.5, 11.5, 1.5]","[nan, nan, nan]","[0.19999999999999996, 1.3, 0.19999999999999996]","[nan, nan, nan]","[0.028428093645484945, 0.11304347826086956, 0....","[nan, nan, nan]",Be,sdOB,RV,[2008ApJ...686.1280P],q errors might be wrong
1,HD200120,"[0.00011299, 314.956476, 0.00011299]","[8.039e-05, 47.520934, 8.039e-05]","[0.0011, 28.1871, 0.0011]","[nan, 0.0, nan]","[1.5500000000000003, 7.85, 1.5500000000000003]","[nan, nan, nan]","[0.14500000000000002, 0.765, 0.14500000000000002]","[nan, nan, nan]","[0.03149478249085243, 0.09745222929936306, 0.0...","[nan, nan, nan]",Be,sdOB,RV,[2013ApJ...765....2P],q errors might be wrong
2,HD10516,"[6.187e-05, 25.915158, 6.187e-05]","[3.664e-05, 50.688731, 3.664e-05]","[0.0035, 126.6982, 0.0035]","[nan, 0.0, nan]","[0.3, 9.6, 0.3]","[nan, nan, nan]","[0.2, 1.2, 0.2]","[nan, nan, nan]","[0.02398989898989899, 0.125, 0.025537634408602...","[nan, nan, nan]",Be,sdOB,RV,[2015A&A...577A..51M],q errors might be wrong
3,HD55606,"[5.81e-06, 108.392085, 5.81e-06]","[5.36e-06, -2.077528, 5.36e-06]","[0.02, 93.76, 0.02]","[nan, 0.0, nan]","[0.2999999999999998, 6.3, 0.2999999999999998]","[nan, nan, nan]","[0.03500000000000003, 0.865, 0.03500000000000003]","[nan, nan, nan]","[0.011544011544011551, 0.1373015873015873, 0.0...","[nan, nan, nan]",Be,sdOB,RV,[2018ApJ...865...76C],q errors might be wrong
4,HD109387,"[0.00018685, 188.370597, 0.00018685]","[8.244e-05, 69.788239, 8.244e-05]","[0.0058, 61.5496, 0.0058]","[nan, 0.0, nan]","[0.48, 3.65, 0.48]","[nan, nan, nan]","[0.043, 0.426, 0.043]","[nan, nan, nan]","[0.023976251285283093, 0.11671232876712329, 0....","[nan, nan, nan]",Be,sdOB,RV,[2022ApJ...940...86K],q errors might be wrong
5,HD113120,"[5.351e-05, 195.772233, 5.351e-05]","[1.806e-05, -71.475734, 1.806e-05]","[0.11, 181.54, 0.11]","[nan, 0.0, nan]","[2.2, 10.1, 2.2]","[nan, nan, nan]","[0.31, 1.43, 0.31]","[nan, nan, nan]","[0.0505272478467359, 0.14158415841584157, 0.07...","[nan, nan, nan]",Be,sdOB,RV,[2023AJ....165..203W],q errors might be wrong
6,HD137387,"[4.868e-05, 232.878426, 4.868e-05]","[1.608e-05, -73.389592, 1.608e-05]","[0.1, 192.1, 0.1]","[nan, 0.0, nan]","[1.0, 11.8, 1.0]","[nan, nan, nan]","[0.14, 1.6, 0.14]","[nan, nan, nan]","[0.021530720338983053, 0.13559322033898305, 0....","[nan, nan, nan]",Be,sdOB,RV,[2023AJ....165..203W],q errors might be wrong
7,HD152478,"[1.587e-05, 254.036842, 1.587e-05]","[7.42e-06, -50.67479, 7.42e-06]","[0.18, 236.5, 0.18]","[nan, 0.0, nan]","[1.3, 6.5, 1.3]","[nan, nan, nan]","[0.11, 0.53, 0.11]","[nan, nan, nan]","[0.027692307692307697, 0.08153846153846155, 0....","[nan, nan, nan]",Be,sdOB,RV,[2023AJ....165..203W],q errors might be wrong
8,HD157042,"[3.028e-05, 260.816984, 3.028e-05]","[1.697e-05, -47.468196, 1.697e-05]","[0.04, 176.17, 0.04]","[nan, 0.0, nan]","[2.9, 10.5, 2.9]","[nan, nan, nan]","[0.29, 1.06, 0.29]","[nan, nan, nan]","[0.043489694385216784, 0.10095238095238096, 0....","[nan, nan, nan]",Be,sdOB,RV,[2023AJ....165..203W],q errors might be wrong
9,HD41335,"[3.051e-05, 91.056251, 3.051e-05]","[3.775e-05, -6.708951, 3.775e-05]","[0.0044, 80.8733, 0.0044]","[nan, 0.0, nan]","[5.7, 17.6, 5.7]","[nan, nan, nan]","[0.22, 1.03, 0.22]","[nan, nan, nan]","[0.023758778774873188, 0.05852272727272727, 0....","[nan, nan, nan]",Be,sdOB,RV,[2024ApJ...962...70K],q errors might be wrong
